In [1]:
using LinearAlgebra, SparseArrays
using Arpack
import gmsh
gmsh.initialize()

include("FEM.jl")
using .FEM

In [2]:
# Adatok
E = 2e5
ν = 0.3
ρ = 7.85e-9
thickness = 1
height = 10
base = 100
elemSize = 2 #22

approxOrder = 2
internalNodes = true
quadElements = true

problem = FEM.Problem("rectangle", type="PlaneStress", E=E, ν=ν, ρ=ρ, b=thickness)

Main.FEM.Problem("rectangle", "PlaneStress", 200000.0, 0.3, 7.85e-9, 1.0)

In [3]:
# Geometria megrajzolása
gmsh.model.add(problem.name)

p1 = gmsh.model.occ.addPoint(0, 0, 0)
p2 = gmsh.model.occ.addPoint(base, 0, 0)
#p23 = gmsh.model.occ.addPoint(base, height / 2, 0)
p3 = gmsh.model.occ.addPoint(base, height, 0)
p4 = gmsh.model.occ.addPoint(0, height, 0)

l1 = gmsh.model.occ.addLine(p1, p2)
l2 = gmsh.model.occ.addLine(p2, p3)
#l23 = gmsh.model.occ.addLine(p23, p3)
l3 = gmsh.model.occ.addLine(p3, p4)
l4 = gmsh.model.occ.addLine(p4, p1)

cl1 = gmsh.model.occ.addCurveLoop([l1, l2, l3, l4])

l5 = gmsh.model.occ.addCircle(base / 2, height / 2, 0, min(base, height) / 4)
cl2 = gmsh.model.occ.addCurveLoop([l5])

sf1 = gmsh.model.occ.addPlaneSurface([cl1, cl2])
#display(sf1)

#cp1 = gmsh.model.occ.addPoint(base / 2, height / 2, 0)
#cl1 = gmsh.model.occ.addLine(p1, p3)
#cl2 = gmsh.model.occ.addLine(p2, p4)

#sf2 = gmsh.model.occ.fragment([(2, sf1)], [(1, cl1)])
#gmsh.model.occ.removeAllDuplicates()

#p5 = gmsh.model.occ.addPoint(0, height / 2, 0)
#p6 = gmsh.model.occ.addPoint(base, height / 2, 0)
#l5 = gmsh.model.occ.addLine(p5, p6)

gmsh.model.occ.synchronize()
#sf2

In [4]:
phg = gmsh.model.addPhysicalGroup(1, [l2])
gmsh.model.setPhysicalName(1, phg, "supp")
phg = gmsh.model.addPhysicalGroup(1, [l4])
gmsh.model.setPhysicalName(1, phg, "load")

In [5]:
supp = FEM.displacementConstraint("supp", ux=0, uy=0)
load = FEM.traction("load", fx=1, fy=0, thickness=thickness)

("load", 1, 0, 1)

In [6]:
FEM.generateMesh(problem, sf1, elemSize, approxOrder=approxOrder, algorithm=6, quadrangle=quadElements, internalNodes=internalNodes)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 80%] Meshing curve 5 (Circle)
Info    : Done meshing 1D (Wall 0.000515684s, CPU 0.000491s)
Info    : 118 nodes 123 elements
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0223246s, CPU 0.022202s)
Info    : 359 nodes 723 elements
Info    : Recombining 2D mesh...
Info    : Blossom: 841 internal 118 closed
Info    : Blossom recombination completed (Wall 0.0145551s, CPU 0.014551s): 294 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.799105, min Q = 0.508151
Info    : Done recombining 2D mesh (Wall 0.0146704s, CPU 0.01467s)
Info    : Meshing order 2 (curvilinear on)...
Info    : [  0%] Meshing curve 1 order 2
Info    : [ 20%] Meshing curve 2 order 2
Info    : [ 40%] Meshing curve 3 order 2
Info    : [ 50%] Meshing

In [7]:
ENV["LIBGL_ALWAYS_SOFTWARE"] = "true"
gmsh.fltk.run()
#gmsh.fltk.wait(3)

-------------------------------------------------------
Version       : 4.12.1-git-38a28aaaf
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20240111
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.7.2
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [8]:
# Néhány láthatósági beállítás 0:kikapcs. 1:bekapcs.
gmsh.option.setNumber("Geometry.CurveLabels", 0)
gmsh.option.setNumber("Geometry.Points", 0)
gmsh.option.setNumber("Geometry.Curves", 0)
gmsh.option.setNumber("Mesh.Points", 0)
gmsh.option.setNumber("Mesh.Lines", 0)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("Mesh.NodeLabels", 0)
gmsh.option.setNumber("Mesh.LineLabels", 0)
gmsh.option.setNumber("Mesh.SurfaceLabels", 0)

In [9]:
K = FEM.stiffnessMatrixPlaneStress(problem)

2588×2588 SparseMatrixCSC{Float64, Int64} with 77152 stored entries:
⎡⠻⣦⣳⣜⠀⠃⠘⣦⢸⢸⠤⢰⢰⢐⣃⣠⡐⣄⠤⠄⠀⠠⣄⢠⡲⠤⠼⠂⠐⡖⣰⢀⠃⠀⠀⠀⠀⡐⡐⡃⎤
⎢⣙⢾⡵⣯⡀⣀⠀⢺⣹⠰⠒⢸⣹⠠⡥⠴⠄⠦⠒⠂⠀⠐⢮⣹⣝⢚⡓⠀⠀⠆⠼⡡⡁⠀⠀⠀⠀⠄⢌⠄⎥
⎢⠤⠀⠀⢨⣻⢞⡳⠆⡧⠂⢌⡧⣻⠁⡗⠴⡣⢚⡁⢈⣹⠦⠀⣉⠀⡇⢈⢈⠀⠀⠁⣟⡷⠌⠀⠀⠀⠐⢻⠀⎥
⎢⠲⣤⣠⣀⠹⠎⠵⣧⢳⣥⣄⣳⣽⣠⣯⣾⣵⣉⣀⡄⠘⢓⣀⣀⣠⣧⣤⣤⠠⣶⣀⣏⢿⣆⣀⠀⢀⠼⠹⡀⎥
⎢⣒⣒⢓⡚⠩⠋⠝⣶⣿⣿⣊⡟⣿⣻⣿⣾⣷⣿⠁⡈⠹⠛⣀⠋⢛⡋⢙⣚⣿⣿⣿⣿⣸⠉⠐⠛⣿⣻⣿⣟⎥
⎢⢀⣃⣘⣀⠦⡵⢤⣹⣮⠼⣻⣾⢿⣿⣭⡉⠌⠠⣟⡛⠦⣴⣂⣶⣾⣿⡛⡿⠤⠤⢢⠲⣘⣶⣾⣻⢯⣏⢇⣅⎥
⎢⢐⢒⠓⡚⠟⠚⠓⣻⣿⣻⣿⣷⣿⣿⡛⣚⣒⣃⠀⡽⢃⡼⣺⢿⡿⣝⣿⣿⣘⣛⣟⣲⣾⣾⢿⣦⣿⣟⣗⣛⎥
⎢⠉⣸⢁⡏⢙⡍⣫⣿⣻⣿⡇⠻⣻⢨⣿⣿⣭⣹⠀⠐⢘⠁⣵⠀⠛⢚⠘⠛⢿⣻⠿⡟⠉⡀⠀⠀⠛⡟⡟⠛⎥
⎢⠐⢬⠠⡅⣩⢊⡕⢻⣽⣿⠂⡁⠼⢸⣇⣻⣿⢟⡀⠀⡨⠑⠈⠀⠀⠐⠀⢀⢇⢿⣪⢫⠴⠂⠀⠀⢉⡲⣝⣙⎥
⎢⠀⠇⠸⠀⡁⢈⠀⠼⡁⠠⣿⠹⣄⡤⢀⠀⠀⠈⣿⣿⡀⠰⠇⢨⠰⡠⣧⡯⠀⠀⡀⡠⠀⢀⢀⣠⡄⢄⡀⢀⎥
⎢⠀⡀⢀⠀⠳⡞⢶⢀⣷⠂⢈⣧⣉⡴⠖⠐⢆⠊⢀⡈⡻⣮⡀⢉⡉⣏⢉⣈⠀⠒⠁⠲⢈⢥⢀⣤⠶⠀⠒⠆⎥
⎢⠀⣙⣎⣳⡄⢠⠀⢸⡤⠘⢨⣼⣾⣞⠑⠛⠂⠀⡉⣁⡄⢈⣻⣾⣾⣤⡄⣨⡠⠣⡙⠙⠰⣲⣖⢀⡐⠀⠀⠃⎥
⎢⠘⡎⣳⢙⠤⠤⠤⣾⡿⠰⣾⣿⣟⢯⣻⢀⢀⠀⠐⡢⡧⢬⠚⣿⣻⣾⣷⣛⢉⣐⣉⠂⢊⠑⠣⠤⠈⢙⠚⡛⎥
⎢⠲⠃⠙⠈⡂⢐⠀⣿⣳⢰⣿⡬⣿⣿⣶⠀⠀⢀⡭⡿⡃⢰⡀⣩⣽⢻⢿⣷⡀⠀⢓⣀⠎⣱⣬⠭⢷⣶⢴⣶⎥
⎢⢰⠤⠠⠄⠀⠀⢠⣦⣿⣿⠀⡇⣶⢸⣿⣳⣭⣕⠀⠀⢠⠀⠤⡊⢃⢰⠀⠈⣿⣿⣼⡍⣈⠂⠀⠀⠙⢶⣷⣅⎥
⎢⠐⢚⠖⡣⣥⢤⡤⢼⣿⣿⢨⡒⢻⣹⣿⠧⡮⣚⠀⡨⢡⡀⣗⠈⠣⠘⠙⢰⡖⠿⣛⣼⣾⣍⠉⠀⣶⡶⣼⡏⎥
⎢⠉⠀⠁⠈⡙⠏⠻⢷⡖⠚⢲⣼⣺⣿⠃⠠⠰⠃⠀⢀⠆⣔⢰⣢⢎⠐⢎⣡⠢⠘⡞⢿⢻⣶⣶⠢⣶⡜⡏⠁⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠘⣴⠀⣾⣻⠻⣷⠀⠀⠀⠀⠀⣰⠀⣴⠘⢙⠉⡆⡆⡟⠀⠀⠃⠀⠸⡛⢻⣶⡆⠀⠄⠀⎥
⎢⢀⠠⠀⠄⢀⠀⣀⡔⣿⣻⡯⢷⣿⢿⣿⠤⢣⡰⠀⢍⠘⠃⠐⠈⣆⢀⢹⣷⢳⣄⢸⡿⣘⠿⠈⠉⣿⣿⣗⣿⎥
⎣⠴⠨⠂⠕⠛⠒⠓⠢⣿⢿⠍⢵⣽⢹⣿⠉⣗⢹⠀⢈⠸⠄⠤⠀⣾⠠⢰⣷⠝⢿⡶⠿⠏⠉⠀⠁⣽⣽⣵⣿⎦

In [10]:
M = FEM.massMatrixPlaneStress(problem)

2588×2588 SparseMatrixCSC{Float64, Int64} with 2588 stored entries:
⎡⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⎦

In [11]:
C = 1e-8 * K

2588×2588 SparseMatrixCSC{Float64, Int64} with 77152 stored entries:
⎡⠻⣦⣳⣜⠀⠃⠘⣦⢸⢸⠤⢰⢰⢐⣃⣠⡐⣄⠤⠄⠀⠠⣄⢠⡲⠤⠼⠂⠐⡖⣰⢀⠃⠀⠀⠀⠀⡐⡐⡃⎤
⎢⣙⢾⡵⣯⡀⣀⠀⢺⣹⠰⠒⢸⣹⠠⡥⠴⠄⠦⠒⠂⠀⠐⢮⣹⣝⢚⡓⠀⠀⠆⠼⡡⡁⠀⠀⠀⠀⠄⢌⠄⎥
⎢⠤⠀⠀⢨⣻⢞⡳⠆⡧⠂⢌⡧⣻⠁⡗⠴⡣⢚⡁⢈⣹⠦⠀⣉⠀⡇⢈⢈⠀⠀⠁⣟⡷⠌⠀⠀⠀⠐⢻⠀⎥
⎢⠲⣤⣠⣀⠹⠎⠵⣧⢳⣥⣄⣳⣽⣠⣯⣾⣵⣉⣀⡄⠘⢓⣀⣀⣠⣧⣤⣤⠠⣶⣀⣏⢿⣆⣀⠀⢀⠼⠹⡀⎥
⎢⣒⣒⢓⡚⠩⠋⠝⣶⣿⣿⣊⡟⣿⣻⣿⣾⣷⣿⠁⡈⠹⠛⣀⠋⢛⡋⢙⣚⣿⣿⣿⣿⣸⠉⠐⠛⣿⣻⣿⣟⎥
⎢⢀⣃⣘⣀⠦⡵⢤⣹⣮⠼⣻⣾⢿⣿⣭⡉⠌⠠⣟⡛⠦⣴⣂⣶⣾⣿⡛⡿⠤⠤⢢⠲⣘⣶⣾⣻⢯⣏⢇⣅⎥
⎢⢐⢒⠓⡚⠟⠚⠓⣻⣿⣻⣿⣷⣿⣿⡛⣚⣒⣃⠀⡽⢃⡼⣺⢿⡿⣝⣿⣿⣘⣛⣟⣲⣾⣾⢿⣦⣿⣟⣗⣛⎥
⎢⠉⣸⢁⡏⢙⡍⣫⣿⣻⣿⡇⠻⣻⢨⣿⣿⣭⣹⠀⠐⢘⠁⣵⠀⠛⢚⠘⠛⢿⣻⠿⡟⠉⡀⠀⠀⠛⡟⡟⠛⎥
⎢⠐⢬⠠⡅⣩⢊⡕⢻⣽⣿⠂⡁⠼⢸⣇⣻⣿⢟⡀⠀⡨⠑⠈⠀⠀⠐⠀⢀⢇⢿⣪⢫⠴⠂⠀⠀⢉⡲⣝⣙⎥
⎢⠀⠇⠸⠀⡁⢈⠀⠼⡁⠠⣿⠹⣄⡤⢀⠀⠀⠈⣿⣿⡀⠰⠇⢨⠰⡠⣧⡯⠀⠀⡀⡠⠀⢀⢀⣠⡄⢄⡀⢀⎥
⎢⠀⡀⢀⠀⠳⡞⢶⢀⣷⠂⢈⣧⣉⡴⠖⠐⢆⠊⢀⡈⡻⣮⡀⢉⡉⣏⢉⣈⠀⠒⠁⠲⢈⢥⢀⣤⠶⠀⠒⠆⎥
⎢⠀⣙⣎⣳⡄⢠⠀⢸⡤⠘⢨⣼⣾⣞⠑⠛⠂⠀⡉⣁⡄⢈⣻⣾⣾⣤⡄⣨⡠⠣⡙⠙⠰⣲⣖⢀⡐⠀⠀⠃⎥
⎢⠘⡎⣳⢙⠤⠤⠤⣾⡿⠰⣾⣿⣟⢯⣻⢀⢀⠀⠐⡢⡧⢬⠚⣿⣻⣾⣷⣛⢉⣐⣉⠂⢊⠑⠣⠤⠈⢙⠚⡛⎥
⎢⠲⠃⠙⠈⡂⢐⠀⣿⣳⢰⣿⡬⣿⣿⣶⠀⠀⢀⡭⡿⡃⢰⡀⣩⣽⢻⢿⣷⡀⠀⢓⣀⠎⣱⣬⠭⢷⣶⢴⣶⎥
⎢⢰⠤⠠⠄⠀⠀⢠⣦⣿⣿⠀⡇⣶⢸⣿⣳⣭⣕⠀⠀⢠⠀⠤⡊⢃⢰⠀⠈⣿⣿⣼⡍⣈⠂⠀⠀⠙⢶⣷⣅⎥
⎢⠐⢚⠖⡣⣥⢤⡤⢼⣿⣿⢨⡒⢻⣹⣿⠧⡮⣚⠀⡨⢡⡀⣗⠈⠣⠘⠙⢰⡖⠿⣛⣼⣾⣍⠉⠀⣶⡶⣼⡏⎥
⎢⠉⠀⠁⠈⡙⠏⠻⢷⡖⠚⢲⣼⣺⣿⠃⠠⠰⠃⠀⢀⠆⣔⢰⣢⢎⠐⢎⣡⠢⠘⡞⢿⢻⣶⣶⠢⣶⡜⡏⠁⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠘⣴⠀⣾⣻⠻⣷⠀⠀⠀⠀⠀⣰⠀⣴⠘⢙⠉⡆⡆⡟⠀⠀⠃⠀⠸⡛⢻⣶⡆⠀⠄⠀⎥
⎢⢀⠠⠀⠄⢀⠀⣀⡔⣿⣻⡯⢷⣿⢿⣿⠤⢣⡰⠀⢍⠘⠃⠐⠈⣆⢀⢹⣷⢳⣄⢸⡿⣘⠿⠈⠉⣿⣿⣗⣿⎥
⎣⠴⠨⠂⠕⠛⠒⠓⠢⣿⢿⠍⢵⣽⢹⣿⠉⣗⢹⠀⢈⠸⠄⠤⠀⣾⠠⢰⣷⠝⢿⡶⠿⠏⠉⠀⠁⣽⣽⣵⣿⎦

In [12]:
K, M, C, f = FEM.applyBoundaryConditions!(problem, K, M, C, [supp], [load]);

In [13]:
Tₘᵢₙ = FEM.smallestPeriodTime(K, M)

2.7452732708927537e-7

In [14]:
q = FEM.solveDisplacement(K, f)

2588×1 Matrix{Float64}:
  0.0005387852269263491
 -7.6115437689389405e-6
  0.0
  0.0
  0.0
  0.0
  0.000538762506926716
  7.3884565755569184e-6
  0.0
  0.0
  ⋮
 -7.853248222284115e-7
  3.1421379658552815e-5
  3.6610079548423785e-6
  2.9191277887710163e-5
 -4.04847997703222e-6
  0.00030929112484203146
 -3.5355086539642655e-6
  0.00022889873554789056
 -3.545778249026863e-6

$$\mathbf{M}\ddot{\mathbf{q}}+\mathbf{C}\dot{\mathbf{q}}+\mathbf{K}\mathbf{q}=\mathbf{f}$$
$$\mathbf{M}\ddot{\mathbf{q}}_n+\mathbf{C}\dot{\mathbf{q}}_n+\mathbf{K}\mathbf{q}_n=\mathbf{f}_n$$
$$\ddot{\mathbf{q}}_n = \frac{\mathbf{q}_{n+1}-2\mathbf{q}_{n}+\mathbf{q}_{n-1}}{\Delta t^2}$$
$$\dot{\mathbf{q}}_n = \frac{\mathbf{q}_{n}-\mathbf{q}_{n-1}}{\Delta t}$$
$$\mathbf{M}\frac{\mathbf{q}_{n+1}-2\mathbf{q}_{n}+\mathbf{q}_{n-1}}{\Delta t^2}+\mathbf{C}\frac{\mathbf{q}_{n}-\mathbf{q}_{n-1}}{\Delta t}+\mathbf{K}\mathbf{q}_n=\mathbf{f}_n$$
$$\mathbf{M}(\mathbf{q}_{n+1}-2\mathbf{q}_{n}+\mathbf{q}_{n-1})+\Delta t\mathbf{C}(\mathbf{q}_{n}-\mathbf{q}_{n-1})+\Delta t^2\mathbf{K}\mathbf{q}_n=\Delta t^2\mathbf{f}_n$$
$$\mathbf{M}\mathbf{q}_{n+1}=\left(2\mathbf{M}-\Delta t\mathbf{C}-\Delta t^{2}\mathbf{K}\right)\mathbf{q}_{n}+(\Delta t\mathbf{C}-\mathbf{M})\mathbf{q}_{n-1}+\Delta t^{2}\mathbf{f}_{n}$$

In [15]:
c = √(problem.E / problem.ρ)
ξₘₐₓ = 1e-1
β = ξₘₐₓ * Tₘᵢₙ / π
C = β * K
u, v, t = FEM.CDM(K, M, C, base / c * 2, Tₘᵢₙ / π * (√(1 + ξₘₐₓ^2) - ξₘₐₓ) * 0.9)
size(u)

(2588, 557)

In [16]:
S = FEM.solveStressPlaneStress(problem, q)

"elemTags = Vector{UInt64}[[0x000000000000007c, 0x000000000000007d, 0x000000000000007e, 0x000000000000007f, 0x0000000000000080, 0x0000000000000081, 0x0000000000000082, 0x0000000000000083, 0x0000000000000084, 0x0000000000000085, 0x0000000000000086, 0x0000000000000087, 0x0" ⋯ 5367 bytes ⋯ "00000194, 0x0000000000000195, 0x0000000000000196, 0x0000000000000197, 0x0000000000000198, 0x0000000000000199, 0x000000000000019a, 0x000000000000019b, 0x000000000000019c, 0x000000000000019d, 0x000000000000019e, 0x000000000000019f, 0x00000000000001a0, 0x00000000000001a1]]"

([[-0.9663738559958264, 0.036019482518845396, 0.0, 0.036019482518845396, -0.07407900603346898, 0.0, 0.0, 0.0, 0.0, -1.0354483194027035  …  0.0, -0.9668919524784818, 0.08222141866118841, 0.0, 0.08222141866118841, -0.06944499369215112, 0.0, 0.0, 0.0, 0.0], [-0.9663975402210722, -0.03620082482717023, 0.0, -0.03620082482717023, -0.07394270689526714, 0.0, 0.0, 0.0, 0.0, -0.8666344017434887  …  0.0, -0.9672985009444018, -0.08230552506090555, 0.0, -0.08230552506090555, -0.06929267675986953, 0.0, 0.0, 0.0, 0.0], [-0.9999999984234273, 7.77702388332208e-9, 0.0, 7.77702388332208e-9, -3.4356889555553494e-10, 0.0, 0.0, 0.0, 0.0, -1.000000008850516  …  0.0, -1.0000000054696279, 1.71188851022092e-9, 0.0, 1.71188851022092e-9, -6.326500601172569e-9, 0.0, 0.0, 0.0, 0.0], [-0.8796445431632653, 0.0689614230525848, 0.0, 0.0689614230525848, -0.1395809017410744, 0.0, 0.0, 0.0, 0.0, -0.9634592443352236  …  0.0, -0.9671010679707414, 0.07171717379132521, 0.0, 0.07171717379132521, -0.06674995118174032, 0.0, 0.0,

In [17]:
gmsh.option.setNumber("View.IntervalsType", 3)
gmsh.option.setNumber("View.VectorType", 5)
gmsh.option.setString("View.Format", "%.6g")

In [18]:
uvec = FEM.showResultUvec(problem, q, name="uvec", visible=false)
display("uvec done")
ux = FEM.showResultUX(problem, q, name="ux", visible=false)
display("ux done")
uy = FEM.showResultUY(problem, q, name="uy", visible=false)
display("uy done")
s = FEM.showResultS(problem, S, name="σ", visible=false, smooth=false)
display("s done")
sx = FEM.showResultSX(problem, S, name="σx", visible=false, smooth=false)
display("sx done")
sy = FEM.showResultSY(problem, S, name="σy", visible=false, smooth=false)
display("sy done")
sxy = FEM.showResultSXY(problem, S, name="τxy", visible=false, smooth=false)
display("sxy done")
vvec = FEM.showResultsVTvec(problem, v, t, name="v(t)", visible=true)
display("vvec done")
#vvec = FEM.showResultUvec(problem, v[:, 1000], name="v(500)", visible=true)

"uvec done"

"ux done"

"uy done"

"s done"

"numElem = [124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, " ⋯ 939 bytes ⋯ " 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417]"

...

294-element Vector{Any}:
 [-0.9663738559958264, -1.0354483194027035, -1.0579654261138334, -0.8659990833099247, -0.9882048298179733, -1.0458003455904505, -0.9465584462469533, -0.9159585949354518, -0.9668919524784818]
 [-0.9663975402210722, -0.8666344017434887, -1.0582735708332192, -1.0355195890078182, -0.9163155226690748, -0.9471970757061566, -1.0459996688191084, -0.9883425272588795, -0.9672985009444018]
 [-0.9999999984234273, -1.000000008850516, -1.0000000090156058, -0.9999999993386782, -1.0000000043972117, -1.0000000085612237, -1.0000000073130162, -0.9999999987416366, -1.0000000054696279]
 [-0.8796445431632653, -0.9634592443352236, -1.0191234104423519, -1.0527407978854515, -0.9181239020047104, -0.9871658357827892, -1.0383213858590592, -0.9490829169720179, -0.9671010679707414]
 [-0.8801542074254203, -1.0533101976898838, -1.0193088960518664, -0.963551594090339, -0.9502156144892356, -1.038710209400723, -0.9873738553837464, -0.9184581420023192, -0.9678218924088737]
 [-1.5954850603594946, 

"sx done"

"sy done"

"sxy done"

"vvec done"

In [19]:
pp1 = gmsh.model.occ.addPoint(0, height / 2, 0)
pp2 = gmsh.model.occ.addPoint(base, height / 2, 0)
pp3 = gmsh.model.occ.addPoint(base / 8, height / 4, 0)

pl1 = gmsh.model.occ.addLine(pp1, pp2)
pl2 = gmsh.model.occ.addSpline([pp1, pp3, pp2])


gmsh.model.occ.synchronize()

In [20]:
sts = ceil(Int64, (base / c * 2) / 6 / (Tₘᵢₙ / π * (√(1 + ξₘₐₓ^2) - ξₘₐₓ)))
display(sts)
Sp = FEM.solveStressPlaneStress(problem, u[:, sts])
sp = FEM.showResultS(problem, Sp, name="σ at t", visible=false, smooth=false)
FEM.plotOnPath(problem, pl1, sp, 500, name="σred on path", visible=false);

84

"elemTags = Vector{UInt64}[[0x000000000000007c, 0x000000000000007d, 0x000000000000007e, 0x000000000000007f, 0x0000000000000080, 0x0000000000000081, 0x0000000000000082, 0x0000000000000083, 0x0000000000000084, 0x0000000000000085, 0x0000000000000086, 0x0000000000000087, 0x0" ⋯ 5367 bytes ⋯ "00000194, 0x0000000000000195, 0x0000000000000196, 0x0000000000000197, 0x0000000000000198, 0x0000000000000199, 0x000000000000019a, 0x000000000000019b, 0x000000000000019c, 0x000000000000019d, 0x000000000000019e, 0x000000000000019f, 0x00000000000001a0, 0x00000000000001a1]]"

In [21]:
#gmsh.option.setNumber("PostProcessing.Binary", 1)
#gmsh.option.setNumber("PostProcessing.SaveMesh", 1)

#gmsh.write("rect.msh")
#gmsh.write("rect.opt")
#gmsh.view.write(4, "rect.pos")

In [22]:
ENV["LIBGL_ALWAYS_SOFTWARE"] = "false"
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.12.1-git-38a28aaaf
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20240111
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.7.2
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [23]:
gmsh.finalize()